In [ ]:
# Cell 1: Install Dependencies
!pip install -q numpy==1.26.4 numba==0.59.1
!pip install -q librosa mediapipe opencv-python-headless pydub moviepy

from google.colab import drive
drive.mount('/content/drive')

print(" Setup complete. Please RESTART RUNTIME before proceeding.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
numba-cuda 0.19.1 requires numba>=0.60.0, but you have numba 0.59.1 which is incompatible.
cuml-cu12 25.10.0 requires numba<0.62.0a0,>=0.60.0, but you have numba 0.59.1 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headle

In [ ]:
# Cell 2: Project Configuration
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

class ProjectConfig:
    """Centralized configuration management"""

    def __init__(self, project_name='AFML_MINIPROJECT'):
        self.PROJECT_NAME = project_name
        self.DRIVE_ROOT = '/content/drive/My Drive'
        self.PROJECT_FOLDER = Path(self.DRIVE_ROOT) / self.PROJECT_NAME

        # Directory structure
        self.DIRS = {
            'data': self.PROJECT_FOLDER / 'data',
            'curated': self.PROJECT_FOLDER / 'data' / 'curated_dataset',
            'features': self.PROJECT_FOLDER / 'data' / 'features',
            'models': self.PROJECT_FOLDER / 'models',
            'demos': self.PROJECT_FOLDER / 'demos',
            'utils': self.PROJECT_FOLDER / 'utils'
        }

        # Create all directories
        for dir_path in self.DIRS.values():
            dir_path.mkdir(parents=True, exist_ok=True)

    # Model parameters
    MODEL_PARAMS = {
        'min_rms': 0.00,
        'max_rms': 0.20,
        'min_db': -40.0,
        'max_db': +10.0
    }

    # Detection thresholds for multi-tap
    MULTITAP_PARAMS = {
        'min_velocity_threshold': 0.02,
        'min_frames_between_taps': 10,
        'impact_window_frames': 5
    }

# Initialize config
config = ProjectConfig()
print("Configuration loaded")
print(f"📁 Project folder: {config.PROJECT_FOLDER}")

✅ Configuration loaded
📁 Project folder: /content/drive/My Drive/AFML_MINIPROJECT


In [ ]:
# Cell 3: Audio Processing Utilities
import librosa
from pydub import AudioSegment

class AudioProcessor:
    """Handle all audio-related operations"""

    @staticmethod
    def extract_audio_features(video_path):
        """Extract RMS and onset time from video"""
        try:
            y, sr = librosa.load(str(video_path), sr=None)

            # RMS Energy
            rms = librosa.feature.rms(y=y)[0]
            peak_rms = float(np.max(rms))

            # Onset detection
            onset_frames = librosa.onset.onset_detect(
                y=y, sr=sr, units='frames',
                wait=1, pre_avg=1, post_avg=1, post_max=1,
                delta=0.02, backtrack=False
            )

            first_onset_ms = float(librosa.frames_to_time(
                onset_frames[0], sr=sr
            ) * 1000) if len(onset_frames) > 0 else 0.0

            return {
                'onset_ms': first_onset_ms,
                'peak_rms': peak_rms,
                'onset_frames': onset_frames.tolist(),
                'success': True
            }
        except Exception as e:
            return {'success': False, 'error': str(e)}

    @staticmethod
    def create_base_sound(video_path, output_path):
        """Extract audio from video to create base sound"""
        try:
            audio = AudioSegment.from_file(str(video_path))
            audio.export(str(output_path), format="wav")
            return True
        except Exception as e:
            print(f" Error creating base sound: {e}")
            return False

    @staticmethod
    def apply_volume_change(audio_path, db_gain):
        """Apply volume change to audio file"""
        audio = AudioSegment.from_file(str(audio_path))
        return audio + db_gain

print(" Audio utilities loaded")

✅ Audio utilities loaded


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
# Cell 4: Video Processing Utilities
import cv2
import mediapipe as mp

class VideoProcessor:
    """Handle all video tracking operations"""

    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands_model = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.WRIST_LANDMARK = self.mp_hands.HandLandmark.WRIST

    def extract_trajectory(self, video_path):
        """Extract hand trajectory from video"""
        trajectory_data = []
        cap = cv2.VideoCapture(str(video_path))

        if not cap.isOpened():
            return None, "Video Error"

        frame_number = 0
        fps = cap.get(cv2.CAP_PROP_FPS)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.hands_model.process(image_rgb)

            norm_x, norm_y = None, None
            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                wrist = hand_landmarks.landmark[self.WRIST_LANDMARK]
                norm_x = float(wrist.x)
                norm_y = float(wrist.y)

            trajectory_data.append({
                'frame': frame_number,
                'time_ms': (frame_number / fps) * 1000,
                'x': norm_x,
                'y': norm_y
            })
            frame_number += 1

        cap.release()
        return trajectory_data, f"Processed {frame_number} frames"

    def __del__(self):
        if hasattr(self, 'hands_model'):
            self.hands_model.close()

print(" Video utilities loaded")

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


✅ Video utilities loaded


In [ ]:
# Cell 5: Kinematics Analysis
class KinematicsAnalyzer:
    """Analyze motion and detect impacts"""

    @staticmethod
    def calculate_basic_kinematics(trajectory_data):
        """Calculate velocity and deceleration from trajectory"""
        df = pd.DataFrame(trajectory_data)

        # Interpolate missing values
        df['x'] = df['x'].interpolate(method='linear')
        df['y'] = df['y'].interpolate(method='linear')
        df = df.dropna()

        if df.empty:
            return None

        # Calculate distance between consecutive points
        df['dist'] = np.sqrt(df['x'].diff()**2 + df['y'].diff()**2)

        # Velocity (smoothed)
        df['velocity'] = df['dist'].rolling(
            window=3, min_periods=1, center=True
        ).mean()

        # Acceleration
        df['acceleration'] = df['velocity'].diff().rolling(
            window=3, min_periods=1, center=True
        ).mean()

        return df

    @staticmethod
    def detect_single_impact(kinematics_df):
        """Detect single tap impact"""
        if kinematics_df is None or kinematics_df.empty:
            return None

        max_decel = float(-kinematics_df['acceleration'].min())
        max_decel_idx = kinematics_df['acceleration'].idxmin()

        return {
            'max_deceleration': max_decel,
            'impact_frame': int(kinematics_df.loc[max_decel_idx, 'frame']),
            'impact_time_ms': float(kinematics_df.loc[max_decel_idx, 'time_ms'])
        }

    @staticmethod
    def detect_multiple_impacts(kinematics_df, config):
        """Detect multiple taps in sequence"""
        if kinematics_df is None or kinematics_df.empty:
            return []

        impacts = []
        min_vel_threshold = config.MULTITAP_PARAMS['min_velocity_threshold']
        min_gap = config.MULTITAP_PARAMS['min_frames_between_taps']

        # Find velocity peaks (potential impacts)
        df = kinematics_df.copy()
        df['is_peak'] = (
            (df['velocity'] > min_vel_threshold) &
            (df['velocity'].shift(1) < df['velocity']) &
            (df['velocity'].shift(-1) < df['velocity'])
        )

        peak_indices = df[df['is_peak']].index.tolist()

        # Filter peaks that are too close together
        filtered_peaks = []
        last_peak = -min_gap

        for peak_idx in peak_indices:
            if peak_idx - last_peak >= min_gap:
                filtered_peaks.append(peak_idx)
                last_peak = peak_idx

        # Calculate deceleration for each peak
        for peak_idx in filtered_peaks:
            window_start = max(0, peak_idx - 2)
            window_end = min(len(df), peak_idx + 3)
            window_df = df.iloc[window_start:window_end]

            if not window_df.empty:
                max_decel = float(-window_df['acceleration'].min())
                decel_idx = window_df['acceleration'].idxmin()

                impacts.append({
                    'max_deceleration': max_decel,
                    'impact_frame': int(df.loc[decel_idx, 'frame']),
                    'impact_time_ms': float(df.loc[decel_idx, 'time_ms']),
                    'peak_velocity': float(df.loc[peak_idx, 'velocity'])
                })

        return impacts

print(" Kinematics analyzer loaded")

✅ Kinematics analyzer loaded


In [ ]:
# Cell 6: Model Training
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

class IntensityPredictor:
    """Train and use the PhysiCNet model"""

    def __init__(self, config):
        self.config = config
        self.model = None

    def train_from_summary(self, summary_csv_path):
        """Train model from feature summary CSV"""
        # Load summary
        summary_df = pd.read_csv(summary_csv_path)

        # Build training data
        training_data = []
        for _, row in summary_df.iterrows():
            json_path = row['path_to_json']

            try:
                with open(json_path, 'r') as f:
                    data = json.load(f)

                # Calculate kinematics
                analyzer = KinematicsAnalyzer()
                kinematics_df = analyzer.calculate_basic_kinematics(
                    data['trajectory']
                )

                if kinematics_df is not None:
                    impact = analyzer.detect_single_impact(kinematics_df)

                    if impact:
                        training_data.append({
                            'max_deceleration': impact['max_deceleration'],
                            'peak_rms': data['audio_labels']['peak_rms']
                        })
            except Exception as e:
                print(f"Skipping {row['clip_name']}: {e}")

        # Train model
        train_df = pd.DataFrame(training_data)
        X = train_df[['max_deceleration']].values
        y = train_df['peak_rms'].values

        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.25, random_state=42
        )

        self.model = LinearRegression()
        self.model.fit(X_train, y_train)

        # Evaluate
        if len(X_val) > 0:
            y_pred = self.model.predict(X_val)
            mae = mean_absolute_error(y_val, y_pred)
            r2 = r2_score(y_val, y_pred)

            print(f" Model trained successfully")
            print(f" MAE: {mae:.4f}, R²: {r2:.4f}")

        return self.model

    def save_model(self, path):
        """Save trained model"""
        joblib.dump(self.model, path)
        print(f" Model saved to: {path}")

    def load_model(self, path):
        """Load trained model"""
        self.model = joblib.load(path)
        print(f"Model loaded from: {path}")

    def predict_intensity(self, max_deceleration):
        """Predict RMS intensity from deceleration"""
        if self.model is None:
            raise ValueError("Model not trained or loaded")

        predicted_rms = self.model.predict([[max_deceleration]])[0]
        return predicted_rms

    def rms_to_db(self, predicted_rms):
        """Convert predicted RMS to dB gain"""
        params = self.config.MODEL_PARAMS

        rms = np.clip(predicted_rms, params['min_rms'], params['max_rms'])
        percentage = (rms - params['min_rms']) / (
            params['max_rms'] - params['min_rms']
        )

        db_gain = params['min_db'] + percentage * (
            params['max_db'] - params['min_db']
        )

        return db_gain

print(" Intensity predictor loaded")

✅ Intensity predictor loaded


In [ ]:
# Cell 7: Single-Tap Demo Generator
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

class SingleTapDemo:
    """Generate single-tap demonstrations"""

    def __init__(self, config, predictor):
        self.config = config
        self.predictor = predictor
        self.video_processor = VideoProcessor()
        self.audio_processor = AudioProcessor()
        self.analyzer = KinematicsAnalyzer()

    def generate(self, video_filename, base_sound_path, output_dir):
        """Generate single-tap demo video"""
        print(f"\n🎬 Processing: {video_filename}")

        video_path = self.config.DIRS['curated'] / video_filename

        # Step 1: Extract trajectory
        trajectory, msg = self.video_processor.extract_trajectory(video_path)
        if trajectory is None:
            print(f" Tracking failed: {msg}")
            return None

        # Step 2: Calculate kinematics
        kinematics_df = self.analyzer.calculate_basic_kinematics(trajectory)
        impact = self.analyzer.detect_single_impact(kinematics_df)

        if not impact:
            print(" No impact detected")
            return None

        print(f" Max Deceleration: {impact['max_deceleration']:.4f}")

        # Step 3: Predict intensity
        predicted_rms = self.predictor.predict_intensity(
            impact['max_deceleration']
        )
        db_gain = self.predictor.rms_to_db(predicted_rms)

        print(f" Predicted RMS: {predicted_rms:.4f} → {db_gain:.2f} dB")

        # Step 4: Get sync time
        audio_features = self.audio_processor.extract_audio_features(video_path)
        onset_time_sec = audio_features['onset_ms'] / 1000.0

        print(f" Sync time: {audio_features['onset_ms']:.2f} ms")

        # Step 5: Assemble video
        video_clip = VideoFileClip(str(video_path)).without_audio()
        base_audio = AudioFileClip(str(base_sound_path))

        volume_multiplier = 10**(db_gain / 20.0)
        final_audio = base_audio.volumex(volume_multiplier).set_start(onset_time_sec)

        video_clip.audio = CompositeAudioClip([final_audio])

        # Save output
        output_filename = f"SINGLE_{Path(video_filename).stem}.mp4"
        output_path = output_dir / output_filename

        video_clip.write_videofile(
            str(output_path),
            codec='libx264',
            audio_codec='aac',
            logger=None
        )

        print(f" Saved: {output_path}")

        return {
            'output_path': output_path,
            'impact': impact,
            'predicted_rms': predicted_rms,
            'db_gain': db_gain
        }

print("Single-tap demo generator loaded")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



✅ Single-tap demo generator loaded


In [ ]:
# Cell 8: Multi-Tap Demo Generator
class MultiTapDemo:
    """Generate multi-tap demonstrations"""

    def __init__(self, config, predictor):
        self.config = config
        self.predictor = predictor
        self.video_processor = VideoProcessor()
        self.audio_processor = AudioProcessor()
        self.analyzer = KinematicsAnalyzer()

    def generate(self, video_filename, base_sound_path, output_dir):
        """Generate multi-tap demo video"""
        print(f"\n Processing Multi-Tap: {video_filename}")

        video_path = self.config.DIRS['curated'] / video_filename

        # Step 1: Extract trajectory
        trajectory, msg = self.video_processor.extract_trajectory(video_path)
        if trajectory is None:
            print(f"Tracking failed: {msg}")
            return None

        # Step 2: Calculate kinematics
        kinematics_df = self.analyzer.calculate_basic_kinematics(trajectory)
        impacts = self.analyzer.detect_multiple_impacts(
            kinematics_df,
            self.config
        )

        if not impacts:
            print(" No impacts detected")
            return None

        print(f" Detected {len(impacts)} impacts")

        # Step 3: Predict intensity for each impact
        impact_sounds = []
        for i, impact in enumerate(impacts):
            predicted_rms = self.predictor.predict_intensity(
                impact['max_deceleration']
            )
            db_gain = self.predictor.rms_to_db(predicted_rms)

            print(f"  Impact {i+1}: Decel={impact['max_deceleration']:.4f}, "
                  f"RMS={predicted_rms:.4f}, Gain={db_gain:.2f} dB, "
                  f"Time={impact['impact_time_ms']:.0f} ms")

            impact_sounds.append({
                'time_sec': impact['impact_time_ms'] / 1000.0,
                'db_gain': db_gain
            })

        # Step 4: Assemble video with multiple sounds
        video_clip = VideoFileClip(str(video_path)).without_audio()
        base_audio = AudioFileClip(str(base_sound_path))

        # Create audio clip for each impact
        audio_clips = []
        for sound in impact_sounds:
            volume_multiplier = 10**(sound['db_gain'] / 20.0)
            audio_clip = base_audio.volumex(volume_multiplier).set_start(
                sound['time_sec']
            )
            audio_clips.append(audio_clip)

        video_clip.audio = CompositeAudioClip(audio_clips)

        # Save output
        output_filename = f"MULTI_{Path(video_filename).stem}.mp4"
        output_path = output_dir / output_filename

        video_clip.write_videofile(
            str(output_path),
            codec='libx264',
            audio_codec='aac',
            logger=None
        )

        print(f" Saved: {output_path}")

        return {
            'output_path': output_path,
            'impacts': impacts,
            'impact_sounds': impact_sounds
        }

print(" Multi-tap demo generator loaded")

✅ Multi-tap demo generator loaded


In [ ]:
# Cell 9: Feature Extraction Pipeline
def extract_all_features(config):
    """Extract features from all videos in curated dataset"""

    print("🔬 Starting Feature Extraction")

    video_processor = VideoProcessor()
    audio_processor = AudioProcessor()

    video_files = list(config.DIRS['curated'].glob('*.mp4'))
    video_files.extend(list(config.DIRS['curated'].glob('*.avi')))
    video_files.extend(list(config.DIRS['curated'].glob('*.mov')))
    video_files.extend(list(config.DIRS['curated'].glob('*.MOV')))

    summary_data = []

    for video_file in video_files:
        print(f"\n {video_file.name}")

        # Extract audio features
        audio_features = audio_processor.extract_audio_features(video_file)
        if not audio_features['success']:
            print(f"   Audio extraction failed")
            continue

        # Extract visual features
        trajectory, msg = video_processor.extract_trajectory(video_file)
        if trajectory is None:
            print(f"   Visual tracking failed")
            continue

        # Save features
        output_data = {
            'clip_name': video_file.name,
            'audio_labels': {
                'onset_ms': audio_features['onset_ms'],
                'peak_rms': audio_features['peak_rms']
            },
            'trajectory': trajectory
        }

        output_filename = video_file.stem + '.json'
        output_path = config.DIRS['features'] / output_filename

        with open(output_path, 'w') as f:
            json.dump(output_data, f, indent=2)

        print(f"   Features saved")

        summary_data.append({
            'clip_name': video_file.name,
            'onset_ms': audio_features['onset_ms'],
            'peak_rms': audio_features['peak_rms'],
            'path_to_json': str(output_path)
        })

    # Save summary CSV
    summary_df = pd.DataFrame(summary_data)
    summary_path = config.DIRS['features'] / '_summary_of_features.csv'
    summary_df.to_csv(summary_path, index=False)

    print(f"\n Extraction complete! Processed {len(summary_df)} videos")
    print(f" Summary saved: {summary_path}")

    return summary_df

# Execute
summary_df = extract_all_features(config)

🔬 Starting Feature Extraction

📹 soft_01_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



  ✅ Features saved

📹 hard_01_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



  ✅ Features saved

📹 hard_02_wood.mp4


...

  ✅ Features saved

📹 soft_13_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



  ✅ Features saved

📹 hard_13_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



  ✅ Features saved

✅ Extraction complete! Processed 26 videos
📄 Summary saved: /content/drive/My Drive/AFML_MINIPROJECT/data/features/_summary_of_features.csv


In [ ]:
# Cell 10: Model Training Pipeline
def train_physicnet_model(config):
    """Train the PhysiCNet model"""

    print(" Starting Model Training")

    summary_path = config.DIRS['features'] / '_summary_of_features.csv'

    if not summary_path.exists():
        print(" Feature summary not found. Run feature extraction first!")
        return None

    predictor = IntensityPredictor(config)
    predictor.train_from_summary(summary_path)

    # Save model
    model_path = config.DIRS['models'] / 'physicnet_model.pkl'
    predictor.save_model(model_path)

    return predictor

# Execute
predictor = train_physicnet_model(config)

  df['x'] = df['x'].interpolate(method='linear')

  df['y'] = df['y'].interpolate(method='linear')

  df['x'] = df['x'].interpolate(method='linear')

  df['y'] = df['y'].interpolate(method='linear')

  df['x'] = df['x'].interpolate(method='linear')

  df['y'] = df['y'].interpolate(method='linear')



🧠 Starting Model Training
✅ Model trained successfully
📊 MAE: 0.0140, R²: 0.8219
💾 Model saved to: /content/drive/My Drive/AFML_MINIPROJECT/models/physicnet_model.pkl


In [ ]:
# Cell 11: Single-Tap Demo Execution
def run_single_tap_demos(config):
    """Run single-tap demonstrations"""

    print("Running Single-Tap Demos")

    # Load model
    model_path = config.DIRS['models'] / 'physicnet_model.pkl'
    predictor = IntensityPredictor(config)
    predictor.load_model(model_path)

    # Create base sound
    base_sound_path = config.DIRS['data'] / 'base_hit.wav'
    source_video = config.DIRS['curated'] / 'hard_06_wood.mp4'  # Adjust filename

    if not base_sound_path.exists():
        AudioProcessor.create_base_sound(source_video, base_sound_path)
        print(f"Base sound created: {base_sound_path}")

    # Create demo generator
    demo_gen = SingleTapDemo(config, predictor)

    # Process videos
    test_videos = ['soft_07_wood.mp4', 'hard_07_wood.mp4']  # Adjust filenames

    results = []
    for video_name in test_videos:
        result = demo_gen.generate(
            video_name,
            base_sound_path,
            config.DIRS['demos']
        )
        if result:
            results.append(result)

    print(f"\n Generated {len(results)} single-tap demos")
    print(f" Output folder: {config.DIRS['demos']}")

    return results

# Execute
single_results = run_single_tap_demos(config)

🎬 Running Single-Tap Demos
📂 Model loaded from: /content/drive/My Drive/AFML_MINIPROJECT/models/physicnet_model.pkl

🎬 Processing: soft_07_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



📊 Max Deceleration: 0.0159
🔊 Predicted RMS: 0.0686 → -22.84 dB
⏱️ Sync time: 58.05 ms


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



✅ Saved: /content/drive/My Drive/AFML_MINIPROJECT/demos/SINGLE_soft_07_wood.mp4

🎬 Processing: hard_07_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



📊 Max Deceleration: 0.0481
🔊 Predicted RMS: 0.1656 → 1.41 dB
⏱️ Sync time: 69.66 ms


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



✅ Saved: /content/drive/My Drive/AFML_MINIPROJECT/demos/SINGLE_hard_07_wood.mp4

✅ Generated 2 single-tap demos
📁 Output folder: /content/drive/My Drive/AFML_MINIPROJECT/demos


In [ ]:
# Cell 12: Multi-Tap Demo Execution
def run_multi_tap_demos(config):
    """Run multi-tap demonstrations"""

    print("Running Multi-Tap Demos")

    # Load model
    model_path = config.DIRS['models'] / 'physicnet_model.pkl'
    predictor = IntensityPredictor(config)
    predictor.load_model(model_path)

    # Use existing base sound
    base_sound_path = config.DIRS['data'] / 'base_hit.wav'

    if not base_sound_path.exists():
        print("Base sound not found. Run single-tap demos first!")
        return []

    # Create demo generator
    demo_gen = MultiTapDemo(config, predictor)

    # Process multi-tap videos (you need to record these)
    test_videos = ['Multitap_test_01.mp4', 'Multitap_test_02.mp4']  # Adjust filenames

    results = []
    for video_name in test_videos:
        video_path = config.DIRS['curated'] / video_name
        if video_path.exists():
            result = demo_gen.generate(
                video_name,
                base_sound_path,
                config.DIRS['demos']
            )
            if result:
                results.append(result)
        else:
            print(f" Video not found: {video_name}")

    print(f"\nGenerated {len(results)} multi-tap demos")
    print(f" Output folder: {config.DIRS['demos']}")

    return results

# Execute
multi_results = run_multi_tap_demos(config)

🎬🎬🎬 Running Multi-Tap Demos
📂 Model loaded from: /content/drive/My Drive/AFML_MINIPROJECT/models/physicnet_model.pkl

🎬🎬🎬 Processing Multi-Tap: Multitap_test_01.mp4
🎯 Detected 15 impacts
  Impact 1: Decel=0.0000, RMS=-0.0001, Gain=-40.00 dB, Time=667 ms
  Impact 2: Decel=0.0038, RMS=0.0156, Gain=-36.10 dB, Time=1500 ms
  Impact 3: Decel=0.0040, RMS=0.0162, Gain=-35.94 dB, Time=1933 ms
  Impact 4: Decel=0.0046, RMS=0.0186, Gain=-35.35 dB, Time=2467 ms
  Impact 5: Decel=0.0026, RMS=0.0106, Gain=-37.34 dB, Time=2867 ms
  Impact 6: Decel=0.0000, RMS=-0.0001, Gain=-40.00 dB, Time=3600 ms
  Impact 7: Decel=0.0021, RMS=0.0086, Gain=-37.85 dB, Time=4733 ms
  Impact 8: Decel=0.0040, RMS=0.0165, Gain=-35.88 dB, Time=5133 ms
  Impact 9: Decel=0.0020, RMS=0.0080, Gain=-37.99 dB, Time=5633 ms
  Impact 10: Decel=0.0002, RMS=0.0007, Gain=-39.83 dB, Time=5967 ms
  Impact 11: Decel=-0.0000, RMS=-0.0001, Gain=-40.00 dB, Time=6367 ms
  Impact 12: Decel=0.0049, RMS=0.0200, Gain=-35.00 dB, Time=6900 ms
  I

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



✅ Saved: /content/drive/My Drive/AFML_MINIPROJECT/demos/MULTI_Multitap_test_01.mp4

🎬🎬🎬 Processing Multi-Tap: Multitap_test_02.mp4
🎯 Detected 13 impacts
  Impact 1: Decel=0.0175, RMS=0.0714, Gain=-22.15 dB, Time=467 ms
  Impact 2: Decel=0.0059, RMS=0.0240, Gain=-34.00 dB, Time=1100 ms
  Impact 3: Decel=0.0217, RMS=0.0887, Gain=-17.82 dB, Time=2267 ms
  Impact 4: Decel=-0.0007, RMS=-0.0028, Gain=-40.00 dB, Time=3167 ms
  Impact 5: Decel=0.0074, RMS=0.0303, Gain=-32.42 dB, Time=3433 ms
  Impact 6: Decel=0.0093, RMS=0.0380, Gain=-30.50 dB, Time=3967 ms
  Impact 7: Decel=0.0012, RMS=0.0049, Gain=-38.77 dB, Time=4967 ms
  Impact 8: Decel=0.0033, RMS=0.0132, Gain=-36.69 dB, Time=5300 ms
  Impact 9: Decel=0.0284, RMS=0.1161, Gain=-10.98 dB, Time=5733 ms
  Impact 10: Decel=0.0035, RMS=0.0143, Gain=-36.42 dB, Time=6067 ms
  Impact 11: Decel=0.0106, RMS=0.0433, Gain=-29.17 dB, Time=7433 ms
  Impact 12: Decel=0.0048, RMS=0.0194, Gain=-35.15 dB, Time=8300 ms
  Impact 13: Decel=0.0155, RMS=0.0634, 

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



✅ Saved: /content/drive/My Drive/AFML_MINIPROJECT/demos/MULTI_Multitap_test_02.mp4

✅ Generated 2 multi-tap demos
📁 Output folder: /content/drive/My Drive/AFML_MINIPROJECT/demos


In [ ]:
# Cell 13: Master Execution Pipeline
def run_complete_pipeline():
    """Execute the complete PhysiCNet pipeline"""

    print("=" * 60)
    print("PhysiCNet Complete Pipeline")
    print("=" * 60)

    # Initialize
    config = ProjectConfig()

    # Phase 1: Feature Extraction
    print("\n PHASE 1: Feature Extraction")
    summary_df = extract_all_features(config)

    # Phase 2: Model Training
    print("\n PHASE 2: Model Training")
    predictor = train_physicnet_model(config)

    # Phase 3: Single-Tap Demos
    print("\n PHASE 3: Single-Tap Demonstrations")
    single_results = run_single_tap_demos(config)

    # Phase 4: Multi-Tap Demos
    print("\n PHASE 4: Multi-Tap Demonstrations")
    multi_results = run_multi_tap_demos(config)

    # Summary
    print("\n" + "=" * 60)
    print(" Pipeline Complete!")
    print("=" * 60)
    print(f" Extracted features from {len(summary_df)} videos")
    print(f"Generated {len(single_results)} single-tap demos")
    print(f" Generated {len(multi_results)} multi-tap demos")
    print(f"\n All outputs saved to: {config.PROJECT_FOLDER}")

    return {
        'config': config,
        'summary': summary_df,
        'predictor': predictor,
        'single_results': single_results,
        'multi_results': multi_results
    }

# Execute complete pipeline
results = run_complete_pipeline()

  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



🚀 PhysiCNet Complete Pipeline

📊 PHASE 1: Feature Extraction
🔬 Starting Feature Extraction

📹 soft_01_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



  ✅ Features saved

📹 hard_01_wood.mp4


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



...

  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



📊 Max Deceleration: 0.0481
🔊 Predicted RMS: 0.1656 → 1.41 dB
⏱️ Sync time: 69.66 ms


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



✅ Saved: /content/drive/My Drive/AFML_MINIPROJECT/demos/SINGLE_hard_07_wood.mp4

✅ Generated 2 single-tap demos
📁 Output folder: /content/drive/My Drive/AFML_MINIPROJECT/demos

🎬🎬🎬 PHASE 4: Multi-Tap Demonstrations


NameError: name 'run_multi_tap_demos' is not defined

In [ ]:
# Cell: Video Analysis and Metrics Display

import cv2
import numpy as np
import librosa
from pathlib import Path
from IPython.display import Video, display, HTML
import matplotlib.pyplot as plt
from scipy import signal

class VideoMetricsAnalyzer:
    """Analyze and display metrics for generated demo videos"""

    def __init__(self, config):
        self.config = config

    def extract_audio_metrics(self, video_path):
        """Extract comprehensive audio metrics"""
        # Load audio
        y, sr = librosa.load(str(video_path), sr=None)

        # RMS Energy
        rms = librosa.feature.rms(y=y)[0]
        peak_rms = np.max(rms)
        mean_rms = np.mean(rms)

        # Decibels
        peak_db = 20 * np.log10(peak_rms + 1e-10)
        mean_db = 20 * np.log10(mean_rms + 1e-10)

        # Spectral features
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))

        # Zero crossing rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))

        # Find impact moment
        impact_frame = np.argmax(rms)
        impact_time = librosa.frames_to_time(impact_frame, sr=sr)

        return {
            'peak_rms': peak_rms,
            'mean_rms': mean_rms,
            'peak_db': peak_db,
            'mean_db': mean_db,
            'spectral_centroid': spectral_centroid,
            'spectral_rolloff': spectral_rolloff,
            'zero_crossing_rate': zcr,
            'impact_time': impact_time,
            'duration': len(y) / sr,
            'sample_rate': sr
        }

    def extract_video_metrics(self, video_path):
        """Extract video motion metrics"""
        cap = cv2.VideoCapture(str(video_path))

        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0

        # Analyze motion
        prev_frame = None
        motion_scores = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if prev_frame is not None:
                diff = cv2.absdiff(gray, prev_frame)
                motion_score = np.mean(diff)
                motion_scores.append(motion_score)

            prev_frame = gray

        cap.release()

        max_motion = np.max(motion_scores) if motion_scores else 0
        mean_motion = np.mean(motion_scores) if motion_scores else 0
        impact_frame_idx = np.argmax(motion_scores) if motion_scores else 0

        return {
            'fps': fps,
            'frame_count': frame_count,
            'width': width,
            'height': height,
            'duration': duration,
            'max_motion': max_motion,
            'mean_motion': mean_motion,
            'impact_frame': impact_frame_idx
        }

    def create_metrics_overlay(self, video_path, output_path, audio_metrics):
        """Create video with metrics overlay"""
        cap = cv2.VideoCapture(str(video_path))

        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

        # Prepare overlay text
        overlay_lines = [
            f"Peak Intensity: {audio_metrics['peak_db']:.2f} dB",
            f"Mean Intensity: {audio_metrics['mean_db']:.2f} dB",
            f"Impact Time: {audio_metrics['impact_time']*1000:.1f} ms",
            f"Peak RMS: {audio_metrics['peak_rms']:.4f}"
        ]

        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Create semi-transparent overlay
            overlay = frame.copy()

            # Background rectangle
            cv2.rectangle(overlay, (10, 10), (400, 130), (0, 0, 0), -1)
            frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)

            # Add text
            y_offset = 35
            for line in overlay_lines:
                cv2.putText(frame, line, (20, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                y_offset += 25

            out.write(frame)
            frame_idx += 1

        cap.release()
        out.release()

        return output_path

    def display_comprehensive_metrics(self, video_name):
        """Display all metrics and visualizations for a video"""
        video_path = self.config.DIRS['demos'] / video_name

        if not video_path.exists():
            print(f" Video not found: {video_path}")
            return None

        print(f"{'='*70}")
        print(f" COMPREHENSIVE METRICS: {video_name}")
        print(f"{'='*70}\n")

        # Extract metrics
        print("Extracting Audio Metrics...")
        audio_metrics = self.extract_audio_metrics(video_path)

        print(" Extracting Video Metrics...")
        video_metrics = self.extract_video_metrics(video_path)

        # Display Audio Metrics
        print(f"\n{'─'*70}")
        print(" AUDIO INTENSITY METRICS")
        print(f"{'─'*70}")
        print(f"  Peak Intensity:     {audio_metrics['peak_db']:>8.2f} dB")
        print(f"  Mean Intensity:     {audio_metrics['mean_db']:>8.2f} dB")
        print(f"  Peak RMS:           {audio_metrics['peak_rms']:>8.4f}")
        print(f"  Mean RMS:           {audio_metrics['mean_rms']:>8.4f}")
        print(f"  Impact Time:        {audio_metrics['impact_time']*1000:>8.2f} ms")
        print(f"  Spectral Centroid:  {audio_metrics['spectral_centroid']:>8.2f} Hz")
        print(f"  Spectral Rolloff:   {audio_metrics['spectral_rolloff']:>8.2f} Hz")
        print(f"  Zero Crossing Rate: {audio_metrics['zero_crossing_rate']:>8.4f}")

        # Display Video Metrics
        print(f"\n{'─'*70}")
        print("VIDEO METRICS")
        print(f"{'─'*70}")
        print(f"  Resolution:         {video_metrics['width']}x{video_metrics['height']}")
        print(f"  Frame Rate:         {video_metrics['fps']:.2f} fps")
        print(f"  Total Frames:       {video_metrics['frame_count']}")
        print(f"  Duration:           {video_metrics['duration']:.3f} seconds")
        print(f"  Max Motion:         {video_metrics['max_motion']:.2f}")
        print(f"  Mean Motion:        {video_metrics['mean_motion']:.2f}")
        print(f"  Impact Frame:       {video_metrics['impact_frame']}")

        # Classification
        print(f"\n{'─'*70}")
        print(" CLASSIFICATION")
        print(f"{'─'*70}")

        intensity_class = "HARD" if audio_metrics['peak_db'] > -10 else "MEDIUM" if audio_metrics['peak_db'] > -20 else "SOFT"
        print(f"  Intensity Class:    {intensity_class}")
        print(f"  Quality Score:      {min(100, int((audio_metrics['peak_db'] + 30) * 3)):>3d}/100")

        # Create overlay video
        print(f"\n{'─'*70}")
        print("Creating Annotated Video...")
        overlay_path = self.config.DIRS['demos'] / f"ANNOTATED_{video_name}"
        self.create_metrics_overlay(video_path, overlay_path, audio_metrics)
        print(f" Saved: {overlay_path}")

        # Display videos
        print(f"\n{'─'*70}")
        print(" VIDEO PLAYBACK")
        print(f"{'─'*70}\n")

        # Original video
        print("Original Video:")
        display(Video(str(video_path), embed=True, width=640))

        print("\nAnnotated Video (with metrics overlay):")
        display(Video(str(overlay_path), embed=True, width=640))

        print(f"\n{'='*70}\n")

        return {
            'audio': audio_metrics,
            'video': video_metrics,
            'classification': intensity_class,
            'paths': {
                'original': video_path,
                'annotated': overlay_path
            }
        }

# Initialize analyzer
analyzer = VideoMetricsAnalyzer(config)

# Analyze videos - CHANGE VIDEO NAMES HERE
video_files = [
    'SINGLE_soft_07_wood.mp4',
    'SINGLE_hard_07_wood.mp4'
]

all_results = {}
for video_name in video_files:
    result = analyzer.display_comprehensive_metrics(video_name)
    if result:
        all_results[video_name] = result

print(" Analysis Complete!")
print(f" Annotated videos saved in: {config.DIRS['demos']}")

  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



📊 COMPREHENSIVE METRICS: SINGLE_soft_07_wood.mp4

🔊 Extracting Audio Metrics...
🎬 Extracting Video Metrics...

──────────────────────────────────────────────────────────────────────
🔊 AUDIO INTENSITY METRICS
──────────────────────────────────────────────────────────────────────
  Peak Intensity:       -42.56 dB
  Mean Intensity:       -63.29 dB
  Peak RMS:             0.0075
  Mean RMS:             0.0007
  Impact Time:          777.87 ms
  Spectral Centroid:   1789.85 Hz
  Spectral Rolloff:    3086.76 Hz
  Zero Crossing Rate:   0.0202

──────────────────────────────────────────────────────────────────────
🎬 VIDEO METRICS
──────────────────────────────────────────────────────────────────────
  Resolution:         848x480
  Frame Rate:         30.00 fps
  Total Frames:       34
  Duration:           1.133 seconds
  Max Motion:         10.83
  Mean Motion:        4.70
  Impact Frame:       9

──────────────────────────────────────────────────────────────────────
📈 CLASSIFICATION
────────


Annotated Video (with metrics overlay):


  y, sr = librosa.load(str(video_path), sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)





📊 COMPREHENSIVE METRICS: SINGLE_hard_07_wood.mp4

🔊 Extracting Audio Metrics...
🎬 Extracting Video Metrics...

──────────────────────────────────────────────────────────────────────
🔊 AUDIO INTENSITY METRICS
──────────────────────────────────────────────────────────────────────
  Peak Intensity:       -18.33 dB
  Mean Intensity:       -39.17 dB
  Peak RMS:             0.1212
  Mean RMS:             0.0110
  Impact Time:          789.48 ms
  Spectral Centroid:   1233.24 Hz
  Spectral Rolloff:    2474.89 Hz
  Zero Crossing Rate:   0.0213

──────────────────────────────────────────────────────────────────────
🎬 VIDEO METRICS
──────────────────────────────────────────────────────────────────────
  Resolution:         848x480
  Frame Rate:         30.00 fps
  Total Frames:       24
  Duration:           0.800 seconds
  Max Motion:         20.01
  Mean Motion:        6.72
  Impact Frame:       2

──────────────────────────────────────────────────────────────────────
📈 CLASSIFICATION
──────


Annotated Video (with metrics overlay):




✅ Analysis Complete!
📁 Annotated videos saved in: /content/drive/My Drive/AFML_MINIPROJECT/demos
